In [16]:
from SimilarityClustering import SimilarityClustering
import articles_data

from gensim.models import Word2Vec
import numpy as np
import pandas as pd
import cnouns as cn
from pymongo import MongoClient
from datetime import datetime

In [2]:
client = MongoClient('mongodb://localhost:27017/somanews')
client.somanews.authenticate('ssomanews', 'ssomanews1029')
db = client.get_database('somanews')

crawled_collection = db.get_collection('crawledArticles')
clusters_collection = db.get_collection('clusters')

In [3]:
catelist_path = '../datastore/category2.p'
w2v_src_dir = "../datastore/w2v_src"
w2v_path = "../datastore/sejongcorpus_w2v2.p"
corpus_path = "../datastore/corpus2.p"
now = datetime.now()
prefix = int("%.2d%.2d"%(now.month, now.day))
# prefix = 1104
prefix_str = "%d_00_" % prefix

# Doc2Vec

In [5]:
# articles_data.makeDataset(article_colleciotn, w2v_src_dir, corpus_path)

In [6]:
# articles_data.trainWord2Vec(w2v_src_dir, w2v_path)

# Model

In [7]:
# train_df = articles_data.find_recent_articles(crawled_collection, catelist_path)
# sc = SimilarityClustering()
# sc.train("cate", w2v_path, train_df, path="../datastore", prefix=prefix_str)

In [8]:
sc = SimilarityClustering.load(path="../datastore", prefix=prefix_str)

total: 897 , size_1: 663 , countby: 22
distance: 33588.4295401
variance: 4.30525853461
similarity: 79.8501096847
cohesion: 98.985719341


# Save

In [9]:
sc.save_to_db(prefix, clusters_collection)

# Test

In [10]:
sc.print_clusters()

    cluster  cnt  distance  variance  similarity   cohesion
34      261   13  1.019971  0.078459    0.967132  12.326539

Most similar - [날씨] 추위 한풀 꺾인다…낮 최고기온 14~19도

 기온NNG 추위NNG 서울NNP
 기온NNG 아침NNG 영하NNG
 영하NNG 기온NNG 아침NNG

0.993124186993 3755 [날씨] 추위 한풀 꺾인다…낮 최고기온 14~19도
0.991177260876 2980 [날씨] 서울·경기 등 일부 지역 아침비…낮부터 기온 올라
0.985021352768 2751 내일 낮부터 ‘가을 추위’ 풀려
0.972716569901 117 기온 '뚝' 주말내내 매우 추워…영동·경상 비
0.971872925758 772 [날씨] 바닷가 강풍 조심하세요
0.968961119652 449 31일 중부에 비…1일 서울 아침 영하 1도
0.968378901482 1675 두달 일찍 온 동장군… 강원 첫 한파주의보
0.967144012451 940 서울 올가을 첫 얼음… 11월 1일 아침 영하 2도
0.966422319412 1541 서울 영하 2도·강원 한파주의보 “오늘 더 추워요”
0.963579773903 2542 오늘이 더 추웠다…서울 영하 2.7도
0.956474065781 2375 2일도 서울 영하2도… 3일부터 풀려 주말 평년기온
0.953379631042 1836 올 가을 가장 추운 날…서울 영하 2.2도
0.914458990097 261 추위 성큼… 30일 아침 서울 2도
None
------------------------------------------------------------
    cluster  cnt  distance  variance  similarity  cohesion
21      108   13  1.770821  0.136217    0.884505  6.493348

Most simila

In [11]:
sc.print_topics()

 기온NNG 추위NNG 서울NNP
 기온NNG 아침NNG 영하NNG
 영하NNG 기온NNG 아침NNG
------------------------------------------------------------
 민주NNG 안장NNG 남기NNG
 남기NNG 경찰NNG 부검NNG
 남기NNG 부검NNG 영장NNG
------------------------------------------------------------
 미국NNP 시장NNG 달러NNG
 정치NNG 환율NNG 테마주NNP
 하락NNG 정치NNG 코스피NNP
------------------------------------------------------------
 물가NNG 가격NNG 요금NNG
 김장NNG 물가NNG 배추NNG
 가격NNG 인상NNG 수입NNG
------------------------------------------------------------
 의원NNG 지도부NNG 누리NNG
 증인NNG 의원NNG 최순NNP
 대표NNG 의원NNG 당NNG
------------------------------------------------------------
 지역NNG 청약NNG 분양NNG
 아파트NNG 상승NNG 서울NNP
 주택NNG 분양NNG 아파트NNG
------------------------------------------------------------
 협정NNG 체결NNG 정보NNG
 협정NNG 체결NNG 정부NNG
 일본NNP 정상NNG 정부NNG
------------------------------------------------------------
 거부NNG 선고NNG 항소심NNG
 학교NNG 학생NNG 혐의NNG
 무죄NNG 사건NNG 선고NNG
------------------------------------------------------------
 판매NNG 미션NNG 키NNG
 판매NNG 내수NNG 대비NNG
 판매NNG 전년NNG 분

In [12]:
sc.print_centers()

1 [날씨] 추위 한풀 꺾인다…낮 최고기온 14~19도
2 고 백남기 농민 5.18망월동 묘지에 묻힌다. 유족 장례절차 발표
3 해외 IB “한국 정치적 불확실성, 4분기 성장률 둔화시킬 수도”
4 4인가족 전통시장 김장비용 23만6천원…작년보다 45％↑
5 ‘이정현 사퇴’ 격론 벌이다 ‘총리 김병준’ 보도에 당황한 새누리
6 서울 강남4구·경기 과천, ‘분양권 전매‘ 시장 사라진다
7 한-일 국민들 ’상호 불신’ 여전
8 재심서 무죄판결 받은 삼례 3인조 살인사건 검찰 항소포기
9 현대차의 추웠던 가을…10월 내수 4만7186대로 전년 동기 대비 30.4%↓
10 위안부피해 할머니들도 “정권 퇴진” 시국선언


In [14]:
clusters_collection.find_one({"cluster":1104032})

{u'_id': ObjectId('581df3bdd4fdab264ecc9f6f'),
 u'articles': [{u'_id': u'khan_201610282233005',
   u'author': u'\uc774\ud638\uc900\xb7\uc774\ud6a8\uc0c1 \uae30\uc790 hjlee@kyunghyang.com',
   u'cate': u'\uacbd\uc81c',
   u'category': u'\ub9c8\ucf13\xb7\ube44\uc988 > \uacbd\uc81c\uc77c\ubc18',
   u'cluster': 1104032,
   u'content': u'\u318d\uc7a5\ubc14\uad6c\ub2c8 \ubb3c\uac00 \uae09\ub4f1 \u2018\ube44\uc0c1\u2019\n\n                 \u2018\uc7a5\ubc14\uad6c\ub2c8 \ubb3c\uac00\u2019\uac00 \uc2ec\uc0c1\uce58 \uc54a\ub2e4. \uc9c0\ub09c\uc5ec\ub984 \ud3ed\uc5fc \uc5ec\ud30c\ub85c \uac00\uaca9\uc774 \ud3ed\ub4f1\ud55c \ubb34\xb7\ubc30\ucd94 \uac19\uc740 \ub18d\uc0b0\ubb3c\uc740 \ubb3c\ub860 \uc0dd\uc120, \uc721\ub958 \ub4f1\uc758 \ubb3c\uac00\uac00 \ub4e4\uc369\uc774\uace0 \uc788\ub2e4. \uc5ec\uae30\uc5d0 \uc62c \ucd08\ubd80\ud130 \uc774\uc5b4\uc9c4 \uc8fc\ub958\xb7\uac00\uacf5\uc2dd\ud488 \uac00\uaca9 \uc778\uc0c1\ub3c4 \uac00\ub729\uc774\ub098 \uc5b4\ub824\uc6b4 \uc11c\ubbfc \uac00\uacc4\